# Trp-cage WE Optimization

In [1]:
from msm_we import optimization as mo
import numpy as np

## Load haMSM

In [2]:
import pickle

with open('data/pickled_model', 'rb') as inf:
    model = pickle.load(inf)

## Compute discrepancy, variance

In [3]:
steady_state_distribution = model.pSS
n_active_we_bins = 10

In [4]:
discrepancy, variance = mo.solve_discrepancy(
    tmatrix = model.Tmatrix,
    pi = steady_state_distribution,
    B = model.indTargets
)

[10/03/22 17:36:38] INFO     Computing pi matrix                                                 ]8;id=950467;file:///home/jd/msm_we/msm_we/optimization.py\optimization.py]8;;\:]8;id=513864;file:///home/jd/msm_we/msm_we/optimization.py#49\49]8;;\

## Compute WE bin assignments for each MSM microstate

In [5]:
# This is a list with an element for each MSM microbin, which is the integer index of the
#   WE bin it's assigned to.
# In other words, microstate_assignments[microbin_index] == WE bin index of that microbin
microstate_assignments = mo.get_uniform_mfpt_bins(
    variance, discrepancy, steady_state_distribution, n_active_we_bins
)
microstate_assignments

array([6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       6, 0, 0, 0, 6, 0, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 0, 7])

In [6]:
# Add entries for the basis/target states, since MSM-WE sets those as the last two clusters
microstate_assignments = np.concatenate(
    [microstate_assignments, [n_active_we_bins - 2, n_active_we_bins - 1]]
)
microstate_assignments

array([6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       6, 0, 0, 0, 6, 0, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 0, 7, 8, 9])

## Create `OptimizedBinMapper`

In [7]:
base_mapper = model.clusters.bin_mapper
n_pcoord_dims=1

In [8]:
# Create the new bin mapper for WESTPA
we_bin_mapper = mo.OptimizedBinMapper(
    n_active_we_bins,
    # In case the pcoord is extended, this is the original pcoord dimensionality
    n_pcoord_dims,
    # If the pcoord was extended, pcoord boundaries are in the original pcoord space
    model.basis_pcoord_bounds,
    model.target_pcoord_bounds,
    # The original, non-Optimized BinMapper that WESTPA was run with.
    #   Used for stratified clustering
    base_mapper,
    microstate_assignments,
    model.clusters
)

we_bin_mapper

                    INFO     Multiple arguments provided to binmapper initializer, creating new ]8;id=463011;file:///home/jd/msm_we/msm_we/optimization.py\optimization.py]8;;\:]8;id=150651;file:///home/jd/msm_we/msm_we/optimization.py#167\167]8;;\
                             object                                                                                

                    INFO     Clusterer has 72 total clusters (include 1 for basis and 1 for     ]8;id=997044;file:///home/jd/msm_we/msm_we/optimization.py\optimization.py]8;;\:]8;id=548556;file:///home/jd/msm_we/msm_we/optimization.py#227\227]8;;\
                             target)                                                                               

<OptimizedBinMapper at 0x7fdea5f2a6a0 with 10 bins>

In [9]:
we_bin_mapper.microstate_mapper

array([6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       6, 0, 0, 0, 6, 0, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 0, 7, 8, 9])